In [12]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [13]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
            SELECT
                repo.repo_id,
                repo.repo_name,
                date_trunc( 'month', message.msg_timestamp ) AS message_month,
                message.cntrb_id, 
                AVG ( message_analysis.sentiment_score ) AS average_sentiment
            FROM
                repo, message_analysis, message, issue_message_ref, issues 
                where message_analysis.msg_id = message.msg_id 
                and 
                issue_message_ref.issue_id = issues.issue_id  
                AND
                issue_message_ref.msg_id = message.msg_id 
                AND
                issues.repo_id = repo.repo_id 
            GROUP BY
                repo.repo_id,
                repo.repo_name,
                message.msg_timestamp, 
                message.cntrb_id 
            ORDER BY
                average_sentiment DESC,
                repo_id,
                message_month DESC;
    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,repo_id,repo_name,message_month,cntrb_id,average_sentiment
0,26073,bcbio-nextgen,2013-12-01,388869.0,0.998485
1,26165,scikit-learn,2012-03-01,396317.0,0.997980
2,26108,pandas,2013-09-01,394848.0,0.997923
3,26165,scikit-learn,2015-03-01,396317.0,0.997888
4,26112,numpy,2019-05-01,279376.0,0.997719
...,...,...,...,...,...
1334038,26006,matplotlib,2014-07-01,279396.0,-0.985386
1334039,26188,Loris,2017-12-01,389737.0,-0.987205
1334040,26057,dipy,2017-11-01,281217.0,-0.988547
1334041,26112,numpy,2017-07-01,395963.0,-0.988643


repo_id                       int64
repo_name                    object
message_month        datetime64[ns]
cntrb_id                    float64
average_sentiment           float64
dtype: object